In [2]:
%%bash
pip install gdown
rm -rf *
mkdir images
mkdir data
cd data
gdown 'https://drive.google.com/uc?id=1s64DpxbfRPAjmFdx14ji4RkKPU8KlQge'
tar -xvf images_1.5M_250x500.tar.gz
rm -rf images_1.5M_250x500.tar.gz

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Downloading...
From: https://drive.google.com/uc?id=1s64DpxbfRPAjmFdx14ji4RkKPU8KlQge
To: /content/data/images_1.5M_250x500.tar.gz
3.01GB [01:04, 46.5MB/s]


In [3]:
from __future__ import print_function, division

import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import gc


import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 250
        self.img_cols = 500
        self.channels = 1
        self.img_shape = (250, 500, 1)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        self.discriminator.trainable = False

        validity = self.discriminator(img)

        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        train_it = tf.keras.preprocessing.image.DirectoryIterator(
          directory='data', target_size=(250, 500),
          color_mode='grayscale',
          batch_size=batch_size,
          image_data_generator=ImageDataGenerator(),
          shuffle=True
        )

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            X_train,_ = train_it.next()
            if X_train.shape[0] < batch_size:
              train_it = tf.keras.preprocessing.image.DirectoryIterator(
                directory='data', target_size=(250, 500),
                color_mode='grayscale',
                batch_size=batch_size,
                image_data_generator=ImageDataGenerator(),
                shuffle=True
              )
              X_train,_ = train_it.next()

            X_train = X_train / 127.5 - 1.
            X_train = np.expand_dims(X_train, axis=3)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(X_train, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % sample_interval == 0:
                self.sample_images(epoch)

            if epoch % 3000 == 0 and epoch > 0:
                self.generator.save_weights('dados_generator%d.h5' % (epoch))
                self.discriminator.save_weights('dados_discriminator%d.h5' % (epoch))
                gc.collect()

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def new_sample(self):
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        return tf.reshape(self.generator.predict(noise)[0], [250,500])

In [4]:
GAN().train(epochs=10000, batch_size=32)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 125000)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               64000512  
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 64,132,097
Trainable params: 64,132,097
Non-trainable params: 0
____________________________________________

KeyboardInterrupt: ignored